In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect("nominations.db")

In [3]:
conn

In [6]:
schema = conn.execute("pragma table_info(nominations)")

In [7]:
schema.fetchall()

[(0, 'Year', 'INTEGER', 0, None, 0),
 (1, 'Category', 'TEXT', 0, None, 0),
 (2, 'Nominee', 'TEXT', 0, None, 0),
 (3, 'Won', 'INTEGER', 0, None, 0),
 (4, 'Movie', 'TEXT', 0, None, 0),
 (5, 'Character', 'TEXT', 0, None, 0)]

In [11]:
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]

In [12]:
conn.execute('''CREATE TABLE ceremonies (
    id integer primary key,
    Year integer,
    Host text
)''')

In [15]:
conn.executemany('''INSERT INTO ceremonies (Year, Host) VALUES (?,?)''', 
                 years_hosts)

In [16]:
conn.execute("select * from ceremonies limit 10").fetchall()

[(1, 2010, 'Steve Martin'),
 (2, 2009, 'Hugh Jackman'),
 (3, 2008, 'Jon Stewart'),
 (4, 2007, 'Ellen DeGeneres'),
 (5, 2006, 'Jon Stewart'),
 (6, 2005, 'Chris Rock'),
 (7, 2004, 'Billy Crystal'),
 (8, 2003, 'Steve Martin'),
 (9, 2002, 'Whoopi Goldberg'),
 (10, 2001, 'Steve Martin')]

In [17]:
conn.execute("pragma foreign_keys = on")

In [20]:
create_table = '''
    CREATE TABLE nominations_two(
    id integer primary key,
    category text,
    nominee text,
    movie text,
    character text,
    won integer,
    ceremony_id integer,
    FOREIGN KEY (ceremony_id) REFERENCES ceremonies(id)
    );
'''

In [21]:
conn.execute(create_table)

In [22]:
conn.execute("pragma table_info(nominations_two)").fetchall()

[(0, 'id', 'integer', 0, None, 1),
 (1, 'category', 'text', 0, None, 0),
 (2, 'nominee', 'text', 0, None, 0),
 (3, 'movie', 'text', 0, None, 0),
 (4, 'character', 'text', 0, None, 0),
 (5, 'won', 'integer', 0, None, 0),
 (6, 'ceremony_id', 'integer', 0, None, 0)]

In [23]:
transfer = '''
SELECT nominations.category, nominations.nominee, nominations.movie, nominations.character, nominations.won, ceremonies.id
FROM nominations
INNER JOIN ceremonies ON
nominations.year == ceremonies.year
;'''

In [24]:
joined_nominations = conn.execute(transfer).fetchall()

In [29]:
joined_nominations[0]

('Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1)

In [30]:
insert_nom_two = '''
INSERT INTO nominations_two 
(category, nominee, movie, character, won, ceremony_id)
VALUES (?,?,?,?,?,?)
'''

In [31]:
conn.executemany(insert_nom_two,joined_nominations)

In [32]:
conn.execute("select * from nominations_two limit 2").fetchall()

[(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1),
 (2,
  'Actor -- Leading Role',
  'Jeff Bridges',
  'True Grit',
  'Rooster Cogburn',
  0,
  1)]

In [33]:
conn.execute("DROP TABLE nominations")

In [34]:
conn.execute("ALTER TABLE nominations_two RENAME TO  nominations")

In [35]:
create_movies = '''
    CREATE TABLE movies (
    id INTEGER PRIMARY KEY,
    movie TEXT);
'''

create_actors = '''
    CREATE TABLE actors (
    id INTEGER PRIMARY KEY,
    actor TEXT);
'''

create_ma = '''
    CREATE TABLE movies_actors (
    id INTEGER PRIMARY KEY,
    movie_id  INTEGER,
    actor_id INTEGER,
    FOREIGN KEY (movie_id) REFERENCES movies(id),
    FOREIGN KEY (actor_id) REFERENCES actors(id));
'''

In [36]:
conn.execute(create_movies)

In [37]:
conn.execute(create_actors)

In [38]:
conn.execute(create_ma)

In [39]:
conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

[('ceremonies',),
 ('nominations',),
 ('movies',),
 ('actors',),
 ('movies_actors',)]

let's test this really quickly. 

In [44]:
test_query = '''
    SELECT a.actor, a.id, COUNT(ma.movie_id)
    FROM actors as a
    LEFT JOIN  movies_actors as ma
    ON a.id == ma.actor_id
    GROUP BY a.id'''

In [45]:
conn.execute(test_query).fetchall()

[]